In [0]:
fileroot = "clinicaltrial_2021"
dbutils.fs.cp("/FileStore/tables/" + fileroot + ".zip", "file:/tmp/")

Out[114]: True

In [0]:
import os
os.environ['fileroot'] = fileroot

In [0]:
%sh
unzip -d /tmp/ /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [0]:
dbutils.fs.mkdirs("/FileStore/tables/" + fileroot)

Out[117]: True

In [0]:
dbutils.fs.mv("file:/tmp/" + fileroot + ".csv" , "/FileStore/tables/" + fileroot , True)

Out[118]: True

In [0]:
myrdd1 = sc.textFile('FileStore/tables/clinicaltrial_2021')

header = myrdd1.first()

myrdd1 = myrdd1.filter(lambda row: row != header)

In [0]:
myrdd1.distinct().count()

Out[120]: 387261

In [0]:
myrdd1 = sc.textFile('FileStore/tables/clinicaltrial_2021')
myrdd1.take(2)

Out[121]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||']

In [0]:
myrdd2 = myrdd1.keyBy(lambda line: line.split('|')[5])

In [0]:
myrdd2.take(10)

Out[123]: [('Type',
  'Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions'),
 ('Interventional',
  'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||'),
 ('Interventional',
  'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|'),
 ('Interventional',
  'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|'),
 ('Observational',
  'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|'),
 ('Observational [Patient Registry]',
  'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|'),
 ('Interventional',
  'NCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Int

In [0]:
countType = myrdd2.countByKey()
sortedType = sorted(countType.items())

In [0]:
sc.parallelize(sortedType).sortBy(lambda x: x[1], False).collect()

Out[125]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69),
 ('Type', 1)]

In [0]:
myrdd3 = myrdd1.map(lambda line: line.split('|')) \
                    .map(lambda fields: (fields[7]) ) \
                    .filter(lambda field: field != '')

In [0]:
myrdd3.take(10)

Out[127]: ['Conditions',
 'Autistic Disorder,Autism Spectrum Disorder',
 'Diabetes Mellitus',
 'Tuberculosis,Lung Diseases,Pulmonary Disease',
 'Diverticular Diseases,Diverticulum,Diverticulosis',
 'Asthma',
 'Hypoventilation',
 'Lymphoma',
 'Myositis',
 'Diabetes Mellitus']

In [0]:
myrdd4 = myrdd3.flatMap(lambda line: line.split(','))

In [0]:
myrdd4.take(20)

Out[129]: ['Conditions',
 'Autistic Disorder',
 'Autism Spectrum Disorder',
 'Diabetes Mellitus',
 'Tuberculosis',
 'Lung Diseases',
 'Pulmonary Disease',
 'Diverticular Diseases',
 'Diverticulum',
 'Diverticulosis',
 'Asthma',
 'Hypoventilation',
 'Lymphoma',
 'Myositis',
 'Diabetes Mellitus',
 'Hypertension',
 'Periodontal Diseases',
 'Diabetes Mellitus',
 'Appendicitis',
 'Stomach Ulcer']

In [0]:
condi_counts = myrdd4.map(lambda condition : (condition, 1)) \
                          .reduceByKey(lambda c1,c2: c1+c2) \
                          .sortBy(lambda x: x[1], False)

In [0]:
condi_counts.take(5)

Out[131]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
myrdd5 = myrdd1.keyBy(lambda line: line.split('|')[1])

In [0]:
myrdd5.take(2)

Out[133]: [('Sponsor',
  'Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions'),
 ('The University of Hong Kong',
  'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||')]

In [0]:
fileroot1 = "pharma"
dbutils.fs.cp("/FileStore/tables/" + fileroot1 + ".zip", "file:/tmp/")

Out[134]: True

In [0]:
import os
os.environ['fileroot'] = fileroot1

In [0]:
%sh
unzip -d /tmp/ /tmp/$fileroot.zip

Archive:  /tmp/pharma.zip
  inflating: /tmp/pharma.csv         


In [0]:
dbutils.fs.mkdirs("/FileStore/tables/" + fileroot1)

Out[137]: True

In [0]:
dbutils.fs.mv("file:/tmp/" + fileroot1 + ".csv" , "/FileStore/tables/" + fileroot1 , True)

Out[138]: True

In [0]:
clinicaltrial = myrdd1.map(lambda line: line.split('|')) \
                   .map(lambda values: [values[0], values[1], values[2], values[3] \
                          ,values[4], values[5], values[6], values[7], values[8]])


In [0]:
clinicaltrial_rdd = clinicaltrial.keyBy(lambda line: line[1])

In [0]:
pharma = sc.textFile('FileStore/tables/pharma')

In [0]:
pharmardd = pharma.map(lambda line: line.split('\n'))

In [0]:
pharmardd = pharma.map(lambda line: line.replace('"', '').split(',')) \
                                  .map(lambda values: [values[0], values[1], values[31]])

In [0]:
pharmardd1 = pharmardd.keyBy(lambda line: line[1])

In [0]:
pharma_clinicialtrial_RDD = clinicaltrial_rdd.subtractByKey(pharmardd1)

In [0]:
pharma_clinicialtrial_RDD1 = pharma_clinicialtrial_RDD.countByKey()

In [0]:
sorted_pharma_clinicialtrial_RDD1 = sorted(pharma_clinicialtrial_RDD1.items())

In [0]:
sc.parallelize(sorted_pharma_clinicialtrial_RDD1).sortBy(lambda x: x[1], False).collect()

Out[148]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828),
 ('National Taiwan University Hospital', 1814),
 ('Eli Lilly and Company', 1641),
 ('Memorial Sloan Kettering Cancer Center', 1604),
 ('University of California, San Francisco', 1598),
 ('National Institute of Allergy and Infectious Diseases (NIAID)', 1582),
 ('Duke University', 1581),
 ('Stanford University', 1552),
 ('Bayer', 1469),
 ('Seoul National University Hospital', 1465),
 ('Washington University School of Medicine', 1374),
 ('Johns Hopkins University', 1358),
 ('Hospices Civils de Lyon', 1286),
 ('University of Pennsylvania', 1266),
 ('Yale University', 1263),
 ('VA Office of Research and Development', 122

In [0]:
completed_study_RDD1 = clinicaltrial.filter(lambda x:x[4] != '') \
                                    .filter(lambda x: x[2] == 'Completed') \
                                    .map(lambda x: [x[4], x[2]])

In [0]:
completed_study_RDD1.take(5)

Out[150]: [['Jul 2020', 'Completed'],
 ['Jan 2018', 'Completed'],
 ['Dec 2014', 'Completed'],
 ['Jan 2018', 'Completed'],
 ['Jul 2017', 'Completed']]

In [0]:
completed_study_RDD2 = completed_study_RDD1.map(lambda x: x[0]).map(lambda x: x.split(' ')) \
                    .filter(lambda x: x[0]).filter(lambda x: x[1] == '2021')

In [0]:
completed_study_RDD2.take(5)

Out[152]: [['Jan', '2021'],
 ['Jun', '2021'],
 ['Mar', '2021'],
 ['Jan', '2021'],
 ['May', '2021']]

In [0]:
 completed_study_RDD3 = completed_study_RDD2.map(lambda value: [value[0], 1]) \
                                    .reduceByKey(lambda v1, v2: v1+v2) \
                                    .sortBy(lambda x: x[1], False)

In [0]:
completed_study_RDD3.take(10)

Out[154]: [('Mar', 1227),
 ('Jan', 1131),
 ('Jun', 1094),
 ('May', 984),
 ('Apr', 967),
 ('Feb', 934),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]